In [1]:
%load_ext sql

In [2]:
%sql mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes

Check range of dates

In [3]:
%%sql
select distinct order_year, order_month from dbo.dim_date;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


order_year,order_month
2010,12
2011,1
2011,2
2011,3
2011,4
2011,5
2011,6
2011,7
2011,8
2011,9


Aggregated revenue by month in 2011

In [4]:

%%sql
select date_t.order_year as year, date_t.order_month as month, FORMAT(sum(fact.sales_amount), 'N0') as Revenue from dbo.fact_table fact
left join dbo.dim_date date_t
on fact.date_key = date_t.date_key
where date_t.order_year = 2011
group by date_t.order_month, date_t.order_year
order by date_t.order_month asc;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


year,month,Revenue
2011,1,"560,000"
2011,2,"498,063"
2011,3,"683,267"
2011,4,"493,207"
2011,5,"723,334"
2011,6,"691,123"
2011,7,"681,300"
2011,8,"704,805"
2011,9,"1,019,688"
2011,10,"1,070,705"


Which products generated the most revenue in 2011 (TOP 3)

In [5]:


%%sql
SELECT TOP 3 
    p.prod_code,
    FORMAT(SUM(f.sales_amount), 'N0') AS total_sales
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_product p ON f.prod_code = p.prod_code
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
WHERE 
    d.order_year = 2011 and f.sales_amount > 0
GROUP BY 
    p.prod_code
ORDER BY 
    SUM(f.sales_amount) DESC;





 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


prod_code,total_sales
DOT,"181,578"
23843,"168,470"
22423,"146,615"


TOP 3 products (total quantity) in 2011

In [13]:
%%sql
SELECT TOP 3 
    p.prod_code, p.prod_description,
    FORMAT(SUM(f.quantity), 'N0') AS total_quantity
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_product p ON f.prod_code = p.prod_code
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
WHERE 
    d.order_year = 2011 and f.sales_amount > 0
GROUP BY 
    p.prod_code, p.prod_description
ORDER BY 
    SUM(f.quantity) DESC;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


prod_code,prod_description,total_quantity
23843,"PAPER CRAFT , LITTLE BIRDIE","80,995"
23166,MEDIUM CERAMIC TOP STORAGE JAR,"78,033"
22197,SMALL POPCORN HOLDER,"54,183"


Which product categories are best sellers (quantity) in 2011 (TOP 5)

In [14]:
%%sql
SELECT TOP 5 
    p.prod_category,
    FORMAT(SUM(f.quantity), 'N0') AS total_quantity
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_product p ON f.prod_code = p.prod_code
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
WHERE 
    d.order_year = 2011 and f.sales_amount > 0
GROUP BY 
    p.prod_category
ORDER BY 
    SUM(quantity) DESC;


 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


prod_category,total_quantity
Home Accessories,"1,804,795"
Tools,"716,675"
Sports,"631,080"
Electronics,"558,417"
Toys,"524,796"


TOP 3 categories based on revenue in 2011

In [15]:
%%sql
SELECT TOP 3 
    p.prod_category,
    FORMAT(SUM(f.sales_amount), 'N0') AS total_sales
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_product p ON f.prod_code = p.prod_code
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
WHERE 
    d.order_year = 2011 and f.sales_amount > 0
GROUP BY 
    p.prod_category
ORDER BY 
    SUM(f.sales_amount) DESC;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


prod_category,total_sales
Home Accessories,"3,253,767"
Electronics,"1,263,015"
Tools,"1,203,469"


Total sales by weekday

In [16]:
%%sql
SELECT 
    CASE d.order_weekday
        WHEN 0 THEN 'Monday'
        WHEN 1 THEN 'Tuesday'
        WHEN 2 THEN 'Wednesday'
        WHEN 3 THEN 'Thursday'
        WHEN 4 THEN 'Friday'
        WHEN 5 THEN 'Saturday'
        WHEN 6 THEN 'Sunday'
    END AS weekday_name,
    FORMAT(SUM(f.sales_amount), 'N0') AS total_sales
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
WHERE 
    d.order_year = 2011 AND f.sales_amount > 0
GROUP BY 
    d.order_weekday
ORDER BY 
    d.order_weekday;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


weekday_name,total_sales
Monday,"1,659,949"
Tuesday,"1,986,456"
Wednesday,"1,710,152"
Thursday,"2,040,398"
Friday,"1,688,795"
Sunday,"757,189"


Sales per hours

In [17]:
%%sql
SELECT 
    t.order_hour,
    FORMAT(SUM(f.sales_amount), 'N0') AS total_sales
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
JOIN 
    dbo.dim_specific_time t ON f.time_key = t.time_key
WHERE 
    d.order_year = 2011 and f.sales_amount > 0
GROUP BY 
    t.order_hour
ORDER BY 
    t.order_hour;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


order_hour,total_sales
6,4
7,"30,592"
8,"277,618"
9,"927,117"
10,"1,361,449"
11,"1,155,558"
12,"1,357,683"
13,"1,176,920"
14,"1,076,491"
15,"1,175,175"


Total sales by country

In [18]:
%%sql
SELECT TOP 5
    c.client_country,
    FORMAT(SUM(f.sales_amount), 'N0') AS total_sales
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
JOIN 
    dbo.dim_client c ON f.client_id = c.client_id
WHERE 
    d.order_year = 2011 and f.sales_amount > 0
GROUP BY 
    c.client_country
ORDER BY 
    SUM(f.sales_amount) DESC;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


client_country,total_sales
United Kingdom,"8,276,953"
Netherlands,"276,662"
EIRE,"273,421"
Germany,"213,626"
France,"200,099"


Countries with the highest per-client average sales

In [19]:
%%sql
SELECT TOP 5
    client_country,
    FORMAT(AVG(avg_sales_per_client), 'N2') AS avg_sales_per_client
FROM (
    SELECT 
        c.client_id,
        c.client_country,
        AVG(f.sales_amount) OVER (PARTITION BY c.client_id) AS avg_sales_per_client
    FROM 
        dbo.fact_table f
    JOIN 
        dbo.dim_date d ON f.date_key = d.date_key
    JOIN 
        dbo.dim_client c ON f.client_id = c.client_id
    WHERE 
        d.order_year = 2011 AND f.sales_amount > 0
) AS client_sales
GROUP BY 
    client_country
ORDER BY 
    avg_sales_per_client DESC;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


client_country,avg_sales_per_client
Singapore,95.85
Sweden,81.28
Hong Kong,55.25
Denmark,45.92
Lebanon,37.64


Standard deviation of the total daily sales

In [20]:
%%sql
SELECT STDEV(daily.total_sales) AS std_dev_sales 
FROM (
SELECT d.order_day, d.order_month, d.order_year ,SUM(f.sales_amount) AS total_sales
FROM dbo.fact_table f
JOIN dbo.dim_date d ON f.date_key = d.date_key
WHERE d.order_year = 2011 AND f.sales_amount > 0
GROUP BY d.order_day, d.order_month, d.order_year
) AS daily;


 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


std_dev_sales
21056.093233526397


10 days moving average (trend) with residuals (difference between trend and daily total sales)

In [21]:
%%sql
SELECT TOP 20
    d.order_date,
    SUM(f.sales_amount) AS daily_sales,
    AVG(SUM(f.sales_amount)) OVER (
        ORDER BY d.order_date
        ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
    ) AS moving_avg_10_days,
    SUM(f.sales_amount) - AVG(SUM(f.sales_amount)) OVER (
        ORDER BY d.order_date
        ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
    ) AS residual
FROM 
    dbo.fact_table f
JOIN 
    dbo.dim_date d ON f.date_key = d.date_key
WHERE 
    d.order_year = 2011 AND f.sales_amount > 0
GROUP BY 
    d.order_date
ORDER BY 
    d.order_date;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


order_date,daily_sales,moving_avg_10_days,residual
2011-01-04,16026.08,16026.080000,0.00
2011-01-05,32634.47,24330.275000,8304.20
2011-01-06,40382.85,29681.133333,10701.72
2011-01-07,28836.59,29469.997500,-633.41
2011-01-09,15778.20,26731.638000,-10953.44
2011-01-10,24569.07,26371.210000,-1802.14
2011-01-11,68655.59,32411.835714,36243.75
2011-01-12,24693.78,31447.078750,-6753.30
2011-01-13,20624.64,30244.585555,-9619.95
2011-01-14,47576.90,31977.817000,15599.08


TOP 2 product categories with best total sales per month 

In [22]:
%%sql
WITH ranked_categories AS (
    SELECT 
        d.order_year,
        d.order_month,
        p.prod_category,
        SUM(f.sales_amount) AS total_sales,
        RANK() OVER (
            PARTITION BY d.order_year, d.order_month 
            ORDER BY SUM(f.sales_amount) DESC
        ) AS rank
    FROM 
        dbo.fact_table f
    JOIN 
        dbo.dim_product p ON f.prod_code = p.prod_code
    JOIN 
        dbo.dim_date d ON f.date_key = d.date_key
    WHERE 
        d.order_year = 2011 AND f.sales_amount > 0
    GROUP BY 
        d.order_year, d.order_month, p.prod_category
)
SELECT 
    order_year,
    order_month,
    prod_category,
    rank,
    total_sales
FROM 
    ranked_categories
WHERE 
    rank in (1, 2)
ORDER BY 
    order_year, order_month;

 * mssql+pyodbc://@maciek_d/retail_online?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


order_year,order_month,prod_category,rank,total_sales
2011,1,Home Accessories,1,226046.82
2011,1,Tools,2,151336.99
2011,2,Home Accessories,1,174793.97
2011,2,Electronics,2,84613.47
2011,3,Home Accessories,1,242810.00
2011,3,Electronics,2,95978.02
2011,4,Home Accessories,1,172411.68
2011,4,Electronics,2,78527.21
2011,5,Home Accessories,1,250809.78
2011,5,Sports,2,100035.45
